In [1]:
# import all necessary modules

import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

C:\Users\jho\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# read all individual text reviews to a list of lists then convert to a dataframe

filepath = 'C:/Users/jho/Desktop/data_science/op_spam_v1.4'
dict_reviews = []

negative_or_positive = 'negative_polarity'

for polarity in os.listdir(filepath):
    deceptive_or_truthful = 'deceptive_from_MTurk'
    for classification in os.listdir(filepath+'/{}'.format(negative_or_positive)):
        i = 1
        for fold in os.listdir(filepath+'/{}'.format(negative_or_positive)+'/{}'.format(deceptive_or_truthful)):
            for file in os.listdir(filepath+'/{}'.format(negative_or_positive)+'/{}'.format(deceptive_or_truthful)+'/fold{}'.format(i)):
                with open(filepath+'/{}'.format(negative_or_positive)+'/{}'.format(deceptive_or_truthful)+'/fold{}/'.format(i)+file) as f:
                    dict_reviews.append([file, f.readline()])
                    
            i += 1
        
        if len(dict_reviews) < 401:
            deceptive_or_truthful = 'truthful_from_Web'
        elif len(dict_reviews) > 402:
            deceptive_or_truthful = 'truthful_from_TripAdvisor'
            
    negative_or_positive = 'positive_polarity'
    
#print(len(dict_reviews))
#print(dict_reviews)

all_reviews_df = pd.DataFrame(dict_reviews, columns=['index', 'review'])

#print(all_reviews_df)

# add columns for the classification to the all_reviews_df dataframe.

all_reviews_df.insert(2, 'negative', np.nan)
all_reviews_df.insert(3, 'deceptive', np.nan)

#print(all_reviews_df)

for i in range(0, len(all_reviews_df)):
    
    if i < 800:
        all_reviews_df['negative'].loc[i] = '1'
    else:
        all_reviews_df['negative'].loc[i] = '0'
        
    if 400 <= i < 800:
        all_reviews_df['deceptive'].loc[i] = '0'
    elif i >= 1200:
        all_reviews_df['deceptive'].loc[i] = '0'
    else:
        all_reviews_df['deceptive'].loc[i] = '1'
        
print(all_reviews_df)

# create a dataframe with only the reviews

text_only = all_reviews_df['review']

#print(text_only.head())

C:\Users\jho\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                          index  \
0                d_hilton_1.txt   
1               d_hilton_10.txt   
2               d_hilton_11.txt   
3               d_hilton_12.txt   
4               d_hilton_13.txt   
5               d_hilton_14.txt   
6               d_hilton_15.txt   
7               d_hilton_16.txt   
8               d_hilton_17.txt   
9               d_hilton_18.txt   
10              d_hilton_19.txt   
11               d_hilton_2.txt   
12              d_hilton_20.txt   
13               d_hilton_3.txt   
14               d_hilton_4.txt   
15               d_hilton_5.txt   
16               d_hilton_6.txt   
17               d_hilton_7.txt   
18               d_hilton_8.txt   
19               d_hilton_9.txt   
20                d_james_1.txt   
21               d_james_10.txt   
22               d_james_11.txt   
23               d_james_12.txt   
24               d_james_13.txt   
25               d_james_14.txt   
26               d_james_15.txt   
27               d_j

In [3]:
#### for tfidf vectorizer ####

def make_xy(data, tfidf=None):
    #Your code here    
    if tfidf is None:
        tfidf = TfidfVectorizer(min_df=0, ngram_range=(1,3))
    X = tfidf.fit_transform(all_reviews_df['review'])
    X = X.tocsc()  # some versions of sklearn return COO format
    #y1 = (data.negative == '1').values.astype(np.int)
    y = (data.deceptive == '1').values.astype(np.int)
    return X, y
Xtfidf, ytfidf = make_xy(all_reviews_df)

In [4]:
# TFIDF Vectorizer for MultinomialNB and Linear SVM

#tfidfvectorizer = TfidfVectorizer(min_df=0, stop_words='english')
#Xtfidf=tfidfvectorizer.fit_transform(all_reviews_df.review)

ss = ShuffleSplit(n_splits=1, test_size=0.25)

for train_index, test_index in ss.split(Xtfidf):
    
    #print("%s %s" % (train_index, test_index))
    print("%s" % (test_index))
    
    clf = LinearSVC()
    #clf = svm.SVC(kernel='linear')
    #clf = MultinomialNB()
    clf_fit = clf.fit(Xtfidf[train_index], ytfidf[train_index])
    prediction = clf_fit.predict(Xtfidf[test_index])
    
    print(prediction)

    training_accuracy = clf.score(Xtfidf[train_index], ytfidf[train_index])
    testing_accuracy = clf.score(Xtfidf[test_index],ytfidf[test_index])

    print("Accuracy on training data: {:2f}".format(training_accuracy))
    print("Accuracy on test data:     {:2f}".format(testing_accuracy))
    print('')

[ 269   29 1079 1514  728  369  915  456 1158  251 1335 1268 1302 1200  936
  303 1066  681 1130 1348 1125 1154 1213 1274  223  846   46  391  462  484
  836  257 1116 1226 1030  951 1590 1314  926 1024  414  473  980  686  230
 1566  855 1027 1087  886  977  810  293  744   90  783 1388 1443  596 1418
  548  671 1361  683 1487  336  826  351  492 1534  785 1327  528  816 1005
 1164  578 1118  788 1401  281  523  120  111  428  511  427  629  576  322
   41  145  517 1510  882 1342  779  540  640 1223  156   57  695  906  343
  330  774  561   51  741  365 1160 1399 1292  732  690  628 1273  569  651
  672   13 1029  633  508  837  668  797  885 1023 1042 1070 1097 1449  874
  386 1396  989  904  790 1043  724  660  982  600  519 1180  615  104 1589
  262  706 1166  843   32  573 1054  962   25  142  614  421  520  737 1501
 1315 1034  221 1373  404 1067  541  495 1186  417 1549 1251 1241  186 1488
 1560  821  974  699  626 1192  831 1346  209  574  256  901  159  914 1435
  700  599  

In [5]:
## this prints all of the incorrectly classified reviews

wrong_predictions = []

for index in range(len(prediction)):
    if ytfidf[test_index[index]] != prediction[index]:
        wrong_predictions.append([all_reviews_df['index'].loc[test_index[index]], all_reviews_df['review'].loc[test_index[index]], all_reviews_df['deceptive'].loc[test_index[index]]])

wrong_predictions = pd.DataFrame(wrong_predictions)
print(len(wrong_predictions))
print('')
"""
for i in range(len(wrong_predictions)):
    print(wrong_predictions[0].loc[i])
    print(wrong_predictions[1].loc[i])

#print(wrong_predictions)
"""
## this prints all of the correctly classified reviews

right_predictions = []

for index in range(len(prediction)):
    if ytfidf[test_index[index]] == prediction[index]:
        right_predictions.append([all_reviews_df['index'].loc[test_index[index]], all_reviews_df['review'].loc[test_index[index]], all_reviews_df['deceptive'].loc[test_index[index]]])

right_predictions = pd.DataFrame(right_predictions)
print(len(right_predictions))
print('')
"""
for i in range(len(right_predictions)):
    print(right_predictions[0].loc[i])
    print(right_predictions[1].loc[i])

#print(right_predictions)
"""

50

350



'\nfor i in range(len(right_predictions)):\n    print(right_predictions[0].loc[i])\n    print(right_predictions[1].loc[i])\n\n#print(right_predictions)\n'

In [6]:
false_positives = wrong_predictions[wrong_predictions[2] == '0']
false_positives

true_positives = right_predictions[right_predictions[2] == '1']
true_positives

,0,1,2
0,d_knickerbocker_18.txt,When we stayed at the Millennium Knickerbocker...,1
1,d_james_18.txt,I had a business trip coming up in Chicago and...,1
2,d_knickerbocker_9.txt,We stayed at the Millennium Knickerbocker last...,1
5,d_intercontinental_18.txt,"I've never written a review before, but I just...",1
6,d_ambassador_5.txt,O.M.G best hotel ever ! i've stayed at various...,1
8,d_amalfi_8.txt,My wife and me stayed in the Amalfi Hotel Chic...,1
9,d_homewood_2.txt,My husband and I planned a romantic getway for...,1
14,d_hardrock_6.txt,I loved staying at the Hard Rock Hotel in Chic...,1
15,d_swissotel_12.txt,Swissotel Chicago: Don't let the shiny bright ...,1
17,d_allegro_19.txt,If you are traveling to Chicago and need a pl...,1


In [7]:
# get the true positive and false positive features.

tfidf_tp = TfidfVectorizer(min_df=0, ngram_range=(1,2))
tfidf_fp = TfidfVectorizer(min_df=0, ngram_range=(1,2))
csr_mat_tp = tfidf_tp.fit_transform(true_positives[1])
csr_mat_fp = tfidf_fp.fit_transform(false_positives[1])

index_words_tp = tfidf_tp.vocabulary_
index_words_tp_list = list(index_words_tp)
index_words_fp = tfidf_fp.vocabulary_
index_words_fp_list = list(index_words_fp)

tp_averages = csr_mat_tp.todense().mean(axis=0)
fp_averages = csr_mat_fp.todense().mean(axis=0)

In [8]:
# true positive features and their importance

tp_feat_avg = []

for word, col in index_words_tp.items():
    tp_feat_avg.append((tp_averages[0, col], index_words_tp[word]))
    
tp_feat_avg_word = []

for i in range(len(tp_feat_avg)):
    tp_feat_avg_word.append(list((tp_feat_avg[i], index_words_tp_list[i])))
    
tp_feat_avg_word.sort(reverse=True)

for words in range(len(tp_feat_avg_word)):
    print(tp_feat_avg_word[words])

[(0.13706516462638985, 13966), 'the']
[(0.07072432924765952, 717), 'and']
[(0.066226955601568799, 14998), 'to']
[(0.065996561759521302, 16088), 'was']
[(0.041412584061686145, 6993), 'hotel']
[(0.040939878440829927, 16480), 'we']
[(0.039243659152710476, 7367), 'in']
[(0.035347236085937178, 10001), 'of']
[(0.034242378210176637, 9390), 'my']
[(0.030396516362167513, 12069), 'room']
[(0.030385305854758359, 7799), 'it']
[(0.028568730647406584, 5613), 'for']
[(0.026930093863711986, 1591), 'at']
[(0.026704838499166699, 3046), 'chicago']
[(0.025769800723383363, 10538), 'our']
[(0.025332432577701468, 7668), 'is']
[(0.023985498464886731, 14791), 'this']
[(0.023879585531179016, 9860), 'not']
[(0.023402944768371129, 13857), 'that']
[(0.023388357759238886, 17142), 'with']
[(0.021176354540293346, 6391), 'had']
[(0.019993761797462321, 13339), 'stay']
[(0.019823587717723003, 17498), 'you']
[(0.019523334433167124, 14684), 'they']
[(0.01931381267422794, 16721), 'were']
[(0.019286701303264837, 15862), 've

[(0.0015008604409938362, 16175), 'was delicious']
[(0.0014990711548845429, 6306), 'great staff']
[(0.0014985131644302714, 5501), 'first time']
[(0.0014978738965547656, 1814), 'bags and']
[(0.001497174169129334, 4723), 'ended up']
[(0.0014967766614836203, 10585), 'our needs']
[(0.0014948996709823557, 1383), 'are very']
[(0.0014926541457223306, 14387), 'the quality']
[(0.0014912251089987485, 2689), 'but this']
[(0.0014903696433895119, 16891), 'when first']
[(0.0014901494449756686, 8398), 'like royalty']
[(0.001489570571074558, 3981), 'deep']
[(0.0014891923239670731, 17564), 'you would']
[(0.0014887527063303736, 7594), 'instead of']
[(0.001488662542873654, 14368), 'the pool']
[(0.0014884302492336576, 12927), 'smoking room']
[(0.0014881567183100259, 9028), 'member']
[(0.0014828506703936863, 2784), 'came with']
[(0.0014790587251387289, 9576), 'near the']
[(0.0014786540528042473, 5659), 'for hotel']
[(0.0014776320370589908, 11853), 'requested']
[(0.0014765537350394497, 3171), 'choose']
[(0.0

[(0.00079357795099698619, 15531), 'twice and']
[(0.00079357546767705304, 14421), 'the shedd']
[(0.00079357546767705304, 13991), 'the aquarium']
[(0.00079357546767705304, 12626), 'shedd aquarium']
[(0.00079357546767705304, 12625), 'shedd']
[(0.00079357546767705304, 9718), 'nice to']
[(0.00079357546767705304, 1313), 'aquarium package']
[(0.0007934530097342663, 14709), 'they found']
[(0.00079332549712766238, 3296), 'cleaning staff']
[(0.00079332431830683714, 2528), 'bring up']
[(0.00079302963212252132, 14259), 'the linens']
[(0.00079285928689737698, 12754), 'signal']
[(0.00079279070619189004, 16636), 'weekend in']
[(0.00079215019290084258, 1150), 'animals']
[(0.00079200142276415557, 15900), 'very noisy']
[(0.00079200142276415557, 9836), 'noisy']
[(0.00079177264605760763, 7421), 'in it']
[(0.00079146555913590076, 16383), 'was treated']
[(0.00079132914403431595, 2480), 'breakfast and']
[(0.00079125538315176502, 14731), 'they re']
[(0.00079125538315176502, 6345), 'guess']
[(0.000791255383151

[(0.00056819735769238363, 4148), 'did however']
[(0.00056819735769238363, 3891), 'day didn']
[(0.00056819735769238363, 3279), 'clean when']
[(0.00056819735769238363, 3211), 'city has']
[(0.00056819735769238363, 3121), 'chicago they']
[(0.00056819735769238363, 2906), 'center but']
[(0.00056819735769238363, 2501), 'breathtaking looking']
[(0.00056819735769238363, 2461), 'bought my']
[(0.00056819735769238363, 2460), 'bought']
[(0.00056819735769238363, 1486), 'as bought']
[(0.00056819735769238363, 1454), 'arrived as']
[(0.00056819735769238363, 606), 'amazed by']
[(0.00056819059629217893, 13644), 'surprise']
[(0.00056781728024037071, 1282), 'appeared']
[(0.00056762913491426948, 14322), 'the old']
[(0.00056760389716207257, 12806), 'sitting']
[(0.00056759111527578002, 4110), 'desk only']
[(0.00056751071088939045, 3880), 'date']
[(0.00056731524256393171, 7462), 'in some']
[(0.00056726847729213171, 11416), 'promises']
[(0.00056726847729213171, 8168), 'lacking']
[(0.00056687935002742952, 16964),

[(0.00048532644941230615, 2310), 'better you']
[(0.00048532644941230615, 2237), 'believe me']
[(0.00048532644941230615, 1754), 'away with']
[(0.00048532644941230615, 1495), 'as exciting']
[(0.00048532644941230615, 1195), 'any better']
[(0.00048532644941230615, 1103), 'and walk']
[(0.00048532644941230615, 911), 'and late']
[(0.00048532644941230615, 656), 'america chicago']
[(0.00048532644941230615, 655), 'america']
[(0.00048532644941230615, 32), '1900 look']
[(0.00048532644941230615, 31), '1900']
[(0.0004837280191799648, 17572), 'your bill']
[(0.0004837280191799648, 17540), 'you nothing']
[(0.0004837280191799648, 17241), 'with supposed']
[(0.0004837280191799648, 17102), 'will pay']
[(0.0004837280191799648, 16910), 'when stay']
[(0.0004837280191799648, 16860), 'what seems']
[(0.0004837280191799648, 15939), 'view or']
[(0.0004837280191799648, 15704), 'upgrade but']
[(0.0004837280191799648, 15662), 'until you']
[(0.0004837280191799648, 15542), 'two bathrooms']
[(0.0004837280191799648, 1518

[(0.00042028836237468164, 14497), 'the tub']
[(0.00042028836237468164, 14203), 'the hold']
[(0.00042028836237468164, 14116), 'the doors']
[(0.00042028836237468164, 13799), 'terrible service']
[(0.00042028836237468164, 13008), 'soap in']
[(0.00042028836237468164, 12962), 'so don']
[(0.00042028836237468164, 12598), 'shampoo or']
[(0.00042028836237468164, 12517), 'service from']
[(0.00042028836237468164, 12029), 'risk']
[(0.00042028836237468164, 11992), 'reviews wrong']
[(0.00042028836237468164, 11986), 'reviews boy']
[(0.00042028836237468164, 11860), 'required check']
[(0.00042028836237468164, 11859), 'required']
[(0.00042028836237468164, 11711), 'receptionist it']
[(0.00042028836237468164, 11113), 'place ve']
[(0.00042028836237468164, 10840), 'part was']
[(0.00042028836237468164, 10736), 'own risk']
[(0.00042028836237468164, 10480), 'or soap']
[(0.00042028836237468164, 10321), 'once was']
[(0.00042028836237468164, 9725), 'nicest it']
[(0.00042028836237468164, 9724), 'nicest for']
[(0.00

[(0.00037419607874484602, 16438), 'waste money']
[(0.00037419607874484602, 16433), 'wasn working']
[(0.00037419607874484602, 16014), 'waiting for']
[(0.00037419607874484602, 16000), 'wait we']
[(0.00037419607874484602, 15934), 'view and']
[(0.00037419607874484602, 15783), 'used up']
[(0.00037419607874484602, 15769), 'use internet']
[(0.00037419607874484602, 15670), 'up almost']
[(0.00037419607874484602, 15562), 'two times']
[(0.00037419607874484602, 15535), 'twice to']
[(0.00037419607874484602, 15327), 'towels as']
[(0.00037419607874484602, 15237), 'to waste']
[(0.00037419607874484602, 15169), 'to receive']
[(0.00037419607874484602, 15157), 'to people']
[(0.00037419607874484602, 15091), 'to grab']
[(0.00037419607874484602, 14977), 'times because']
[(0.00037419607874484602, 14974), 'timee since']
[(0.00037419607874484602, 14973), 'timee']
[(0.00037419607874484602, 14875), 'though we']
[(0.00037419607874484602, 14607), 'then another']
[(0.00037419607874484602, 14594), 'them then']
[(0.00

[(0.00032105386110100432, 6933), 'home was']
[(0.00032105386110100432, 6639), 'have offered']
[(0.00032105386110100432, 6121), 'glad to']
[(0.00032105386110100432, 6071), 'getting smaller']
[(0.00032105386110100432, 5755), 'former chicagoan']
[(0.00032105386110100432, 5754), 'former']
[(0.00032105386110100432, 5668), 'for maid']
[(0.00032105386110100432, 5646), 'for every']
[(0.00032105386110100432, 5416), 'finally checked']
[(0.00032105386110100432, 5380), 'fi to']
[(0.00032105386110100432, 5301), 'faster internet']
[(0.00032105386110100432, 5061), 'expect to']
[(0.00032105386110100432, 4944), 'every night']
[(0.00032105386110100432, 4900), 'even very']
[(0.00032105386110100432, 4869), 'even faster']
[(0.00032105386110100432, 4636), 'ed was']
[(0.00032105386110100432, 4635), 'ed']
[(0.00032105386110100432, 4544), 'drove away']
[(0.00032105386110100432, 4543), 'drove']
[(0.00032105386110100432, 4478), 'download some']
[(0.00032105386110100432, 4477), 'download']
[(0.0003210538611010043

[(0.00029279630153891505, 1833), 'band']
[(0.00029279630153891505, 1742), 'awake will']
[(0.00029279630153891505, 1628), 'at local']
[(0.00029279630153891505, 1474), 'art deco']
[(0.00029279630153891505, 1284), 'appeared to']
[(0.00029279630153891505, 1230), 'anyone on']
[(0.00029279630153891505, 1092), 'and unique']
[(0.00029279630153891505, 210), 'accurate was']
[(0.00028608825001340439, 17397), 'would certainly']
[(0.00028608825001340439, 17269), 'without ever']
[(0.00028608825001340439, 17228), 'with rather']
[(0.00028608825001340439, 17184), 'with glossy']
[(0.00028608825001340439, 17163), 'with confidence']
[(0.00028608825001340439, 16994), 'while waiting']
[(0.00028608825001340439, 16945), 'which caused']
[(0.00028608825001340439, 16718), 'went up']
[(0.00028608825001340439, 16455), 'water that']
[(0.00028608825001340439, 16453), 'water damage']
[(0.00028608825001340439, 16415), 'washed in']
[(0.00028608825001340439, 16398), 'was visible']
[(0.00028608825001340439, 16317), 'was 

[(0.00018678001031409281, 5077), 'expected luxury']
[(0.00018678001031409281, 4988), 'exactly like']
[(0.00018678001031409281, 4763), 'enough of']
[(0.00018678001031409281, 4672), 'elevator and']
[(0.00018678001031409281, 4596), 'easier after']
[(0.00018678001031409281, 4595), 'easier']
[(0.00018678001031409281, 4463), 'down inside']
[(0.00018678001031409281, 4430), 'door it']
[(0.00018678001031409281, 4305), 'dismay found']
[(0.00018678001031409281, 4304), 'dismay']
[(0.00018678001031409281, 4256), 'disagreement saying']
[(0.00018678001031409281, 4255), 'disagreement']
[(0.00018678001031409281, 4241), 'direct line']
[(0.00018678001031409281, 4157), 'did so']
[(0.00018678001031409281, 4137), 'dial anything']
[(0.00018678001031409281, 4124), 'desperately needed']
[(0.00018678001031409281, 4123), 'desperately']
[(0.00018678001031409281, 4106), 'desk focusing']
[(0.00018678001031409281, 4025), 'delayed and']
[(0.00018678001031409281, 3969), 'decorated hotel']
[(0.00018678001031409281, 378

In [9]:
# false positive features and their importance

fp_feat_avg = []

for word, col in index_words_fp.items():
    fp_feat_avg.append((fp_averages[0, col], index_words_fp[word]))
    
fp_feat_avg_word = []

for i in range(len(fp_feat_avg)):
    fp_feat_avg_word.append(list((fp_feat_avg[i], index_words_fp_list[i])))
    
fp_feat_avg_word.sort(reverse=True)

for words in range(len(fp_feat_avg_word)):
    print(fp_feat_avg_word[words])

[(0.1752201191611672, 4043), 'the']
[(0.094962045276958215, 259), 'and']
[(0.092961984985146295, 4354), 'to']
[(0.085849190332955613, 4686), 'was']
[(0.058266975636714999, 2080), 'in']
[(0.048177855526730699, 2913), 'of']
[(0.045388271963338493, 1978), 'hotel']
[(0.044202037241185141, 4807), 'we']
[(0.043742902383959538, 2228), 'it']
[(0.043208020516280511, 2706), 'my']
[(0.03833413025506998, 3497), 'room']
[(0.035262105616924853, 490), 'at']
[(0.034010640051127312, 2853), 'not']
[(0.032320611767726604, 4008), 'that']
[(0.031513887273416134, 3071), 'our']
[(0.030351395029635405, 4295), 'this']
[(0.029618588686110701, 450), 'as']
[(0.02906756733628401, 908), 'chicago']
[(0.028617460814687981, 2115), 'in the']
[(0.028557294631193098, 1790), 'had']
[(0.028188136792295888, 1577), 'for']
[(0.02739613235570788, 774), 'but']
[(0.027159780077065983, 5090), 'you']
[(0.02572281550637848, 4871), 'were']
[(0.025481279396610105, 3868), 'stay']
[(0.024812283471421285, 1846), 'have']
[(0.024398348492

[(0.0036144856904592034, 2247), 'it looked']
[(0.0036144856904592034, 1992), 'hotel for']
[(0.0036144856904592034, 1866), 'have to']
[(0.0036144856904592034, 1775), 'gross and']
[(0.0036144856904592034, 1708), 'girls weekend']
[(0.0036144856904592034, 1707), 'girls']
[(0.0036144856904592034, 1663), 'furniture was']
[(0.0036144856904592034, 1629), 'friend book']
[(0.0036144856904592034, 1604), 'for us']
[(0.0036144856904592034, 1588), 'for girls']
[(0.0036144856904592034, 1272), 'door man']
[(0.0036144856904592034, 1207), 'didn have']
[(0.0036144856904592034, 923), 'chicago lets']
[(0.0036144856904592034, 860), 'carpet was']
[(0.0036144856904592034, 702), 'book this']
[(0.0036144856904592034, 602), 'be that']
[(0.0036144856904592034, 324), 'and outdated']
[(0.0036144856904592034, 305), 'and just']
[(0.0035792741458032739, 2284), 'itself']
[(0.0035792741458032739, 1189), 'desk was']
[(0.0035622244104339144, 4065), 'the breakfast']
[(0.0035615329194211686, 2651), 'mistake']
[(0.0035615329

[(0.0018329204129580853, 229), 'amazing time']
[(0.0018329204129580853, 218), 'am because']
[(0.0018329204129580853, 189), 'almost everyone']
[(0.0018329204129580853, 156), 'again without']
[(0.0018329204129580853, 124), 'advanced for']
[(0.0018329204129580853, 123), 'advanced']
[(0.0018329204129580853, 113), 'added bonus']
[(0.0018329204129580853, 112), 'added']
[(0.0018329204129580853, 86), 'accidentally discarded']
[(0.0018329204129580853, 85), 'accidentally']
[(0.0018329204129580853, 68), 'about how']
[(0.0018081722733434566, 4466), 'town']
[(0.0018081722733434566, 4351), 'tip']
[(0.0018081722733434566, 4166), 'the phone']
[(0.0018081722733434566, 4023), 'that is']
[(0.0018081722733434566, 4010), 'that are']
[(0.0018081722733434566, 3744), 'smaller boutique']
[(0.0018081722733434566, 3381), 'reason']
[(0.0018081722733434566, 3372), 'real']
[(0.0018081722733434566, 2544), 'manhattan']
[(0.0018081722733434566, 2028), 'hour']
[(0.0018081722733434566, 1991), 'hotel first']
[(0.00180817